In [1]:
import numpy as np
import os
import psycopg2
import pandas as pd
import json
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

In [2]:
#Cloud DB Config
SQL_USERNAME = "postgres" #your personal username goes here
SQL_PASSWORD = "airlineproject" #your personal password goes here
DB_NAME = "airline_db" #The default name of the DB
DB_HOST = "airline-db.c0dqcvhs1y58.us-east-2.rds.amazonaws.com"
PORT = 5432
database_path = f"postgresql://{SQL_USERNAME}:{SQL_PASSWORD}@{DB_HOST}:{PORT}/{DB_NAME}"

In [3]:
from sqlalchemy import create_engine
engine = create_engine(database_path)

In [4]:
#AWS_DB_URL = (
    #os.environ.get('AWS_DB_URL')
    #.replace('postgres://', 'postgresql://', 1)
   # )
#engine = create_engine(AWS_DB_URL)
#connection = engine.connect()

In [5]:
#main_df = pd.read_sql_query('select Dep_Airport, Arr_Airport, Day_Week, month, Mkt_Ccode, Mkt_Fl_No, Arr_Delay_Time_Actual from sample' FETCH FIRST 10 ROWS ONLY, con=engine).head()

In [6]:
#main_df = pd.read_sql_query('select * from sample', con=engine).head()

In [7]:
main_df = pd.read_csv("sample.csv")
main_df.head()

,index,Mkt_Ccode,Mkt_Fl_No,Dep_Airport,Arr_Airport,Date,Day_Week,Sched_Dep_Time_OAG,Gate_Dep_Time_Actual,Sched_Arr_Time_OAG,...,Mins_Late_Delay_Code_G_NAS,Mins_Late_Delay_Code_H_Security,Mins_Late_Delay_Code_I_Late_Arr_Flight,First_Gate_Dep_Time,Total_Ground_Away_From_Gate,Longest_Ground_Time_Away_From_Gate,No_Landings_At_Div_Airport,month,Arr_Delay_Time_Actual_Flag,Sched_Dep_Time_OAG_Block
0,48712,AA,1487.0,ORD,DEN,20180203,sat,2015,2023,2157,...,0,0,0,0,0,0,0,feb,0,evening
1,547480,WN,3389.0,LAS,BUR,20180107,sun,2055,2104,2155,...,0,0,0,0,0,0,0,jan,1,evening
2,592345,AA,4545.0,MIA,BNA,20180111,thu,1110,1108,1240,...,0,0,0,0,0,0,0,jan,0,late_morning
3,29934,AA,1911.0,DFW,PHL,20180220,tue,906,937,1315,...,0,0,0,0,0,0,0,feb,1,late_morning
4,615637,UA,3842.0,ORD,RST,20180130,tue,925,917,1042,...,0,0,0,0,0,0,0,jan,0,late_morning


In [8]:
main_df.drop(["index"], axis = 1, inplace = True)
main_df.head()

,Mkt_Ccode,Mkt_Fl_No,Dep_Airport,Arr_Airport,Date,Day_Week,Sched_Dep_Time_OAG,Gate_Dep_Time_Actual,Sched_Arr_Time_OAG,Gate_Arr_Time_Actual,...,Mins_Late_Delay_Code_G_NAS,Mins_Late_Delay_Code_H_Security,Mins_Late_Delay_Code_I_Late_Arr_Flight,First_Gate_Dep_Time,Total_Ground_Away_From_Gate,Longest_Ground_Time_Away_From_Gate,No_Landings_At_Div_Airport,month,Arr_Delay_Time_Actual_Flag,Sched_Dep_Time_OAG_Block
0,AA,1487.0,ORD,DEN,20180203,sat,2015,2023,2157,2152,...,0,0,0,0,0,0,0,feb,0,evening
1,WN,3389.0,LAS,BUR,20180107,sun,2055,2104,2155,2208,...,0,0,0,0,0,0,0,jan,1,evening
2,AA,4545.0,MIA,BNA,20180111,thu,1110,1108,1240,1225,...,0,0,0,0,0,0,0,jan,0,late_morning
3,AA,1911.0,DFW,PHL,20180220,tue,906,937,1315,1331,...,0,0,0,0,0,0,0,feb,1,late_morning
4,UA,3842.0,ORD,RST,20180130,tue,925,917,1042,1024,...,0,0,0,0,0,0,0,jan,0,late_morning


In [9]:
airports = pd.read_csv("airports.csv")
airport_data = pd.read_csv("airport-codes.csv")
clean_airport_data = airport_data.dropna(how="all")
#clean_airport_data2 = clean_airport_data.drop(labels="Unnamed: 7", axis=1)

In [10]:
main_df2 = main_df.merge(airports, how="left", on="Dep_Airport")

In [11]:
main_df3 = main_df2.merge(clean_airport_data, how="left", on="Dep_Airport")

In [12]:
main_df3.head()

,Mkt_Ccode,Mkt_Fl_No,Dep_Airport,Arr_Airport,Date,Day_Week,Sched_Dep_Time_OAG,Gate_Dep_Time_Actual,Sched_Arr_Time_OAG,Gate_Arr_Time_Actual,...,City name_x,Country name_x,lat,lng,City name_y,Airport Code,Airport Name_y,Country name_y,Country Abbrev.,World Area Code
0,AA,1487.0,ORD,DEN,20180203,sat,2015,2023,2157,2152,...,Chicago,United States,41.978600,-87.904800,Chicago,ORD,Ohare International,United States,US,1
1,WN,3389.0,LAS,BUR,20180107,sun,2055,2104,2155,2208,...,Las Vegas,United States,36.080101,-115.152000,Las Vegas,LAS,Mccarran Intl,United States,US,67
2,AA,4545.0,MIA,BNA,20180111,thu,1110,1108,1240,1225,...,Miami,United States,25.793200,-80.290604,Miami,MIA,Miami International Airport,United States,US,67
3,AA,1911.0,DFW,PHL,20180220,tue,906,937,1315,1331,...,Dallas-Fort Worth,United States,32.896801,-97.038002,Dallas,DFW,Dallas/Ft Worth Intl,United States,US,67
4,UA,3842.0,ORD,RST,20180130,tue,925,917,1042,1024,...,Chicago,United States,41.978600,-87.904800,Chicago,ORD,Ohare International,United States,US,1


In [13]:
main_df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200000 entries, 0 to 199999
Data columns (total 42 columns):
 #   Column                                  Non-Null Count   Dtype  
---  ------                                  --------------   -----  
 0   Mkt_Ccode                               200000 non-null  object 
 1   Mkt_Fl_No                               200000 non-null  float64
 2   Dep_Airport                             200000 non-null  object 
 3   Arr_Airport                             200000 non-null  object 
 4   Date                                    200000 non-null  int64  
 5   Day_Week                                200000 non-null  object 
 6   Sched_Dep_Time_OAG                      200000 non-null  int64  
 7   Gate_Dep_Time_Actual                    200000 non-null  int64  
 8   Sched_Arr_Time_OAG                      200000 non-null  int64  
 9   Gate_Arr_Time_Actual                    200000 non-null  int64  
 10  Sched_Elapsed_Time_CRS                  2000

In [14]:
means = main_df3.groupby('Dep_Airport').mean()

In [15]:
means.head()

,Mkt_Fl_No,Date,Sched_Dep_Time_OAG,Gate_Dep_Time_Actual,Sched_Arr_Time_OAG,Gate_Arr_Time_Actual,Sched_Elapsed_Time_CRS,Actual_Gate_Gate_Time,Dep_Delay_Time_Actual,Arr_Delay_Time_Actual,...,Mins_Late_Delay_Code_G_NAS,Mins_Late_Delay_Code_H_Security,Mins_Late_Delay_Code_I_Late_Arr_Flight,First_Gate_Dep_Time,Total_Ground_Away_From_Gate,Longest_Ground_Time_Away_From_Gate,No_Landings_At_Div_Airport,Arr_Delay_Time_Actual_Flag,lat,lng
Dep_Airport,,,,,,,,,,,,,,,,,,,,,
ABE,4214.654321,2.018014e+07,1188.024691,1220.975309,1307.888889,1323.271605,64.061728,62.296296,25.666667,24.975309,...,2.827160,0.0,11.432099,0.000000,0.000000,0.000000,0.0,0.333333,40.652100,-75.440804
ABQ,3811.127566,2.018014e+07,1206.763930,1170.435484,1412.247801,1368.234604,113.489736,104.719941,3.582111,-1.442815,...,2.979472,0.0,3.788856,0.041056,0.002933,0.002933,0.0,0.259531,35.040199,-106.609001
ABY,3316.384615,2.018021e+07,1081.346154,1117.743590,1184.846154,1226.923077,60.935897,63.538462,31.717949,35.102564,...,3.961538,0.0,25.794872,0.000000,0.000000,0.000000,0.0,0.602564,31.535500,-84.194504
ADK,161.000000,2.018022e+07,1715.000000,1482.142857,2057.000000,1813.714286,162.000000,143.000000,12.142857,16.285714,...,3.571429,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.857143,51.877998,-176.645996
ADQ,43.000000,2.018022e+07,1363.750000,1280.857143,1413.660714,1368.214286,49.910714,50.214286,-7.500000,-5.428571,...,0.553571,0.0,6.553571,0.000000,0.000000,0.000000,0.0,0.178571,57.750000,-152.494003


In [16]:
mean_merge = means.filter(['Dep_Airport', "Arr_Delay_Time_Actual"], axis=1)
mean_merge.rename(columns={"Arr_Delay_Time_Actual": "mean_arr_delay"}, inplace = True)
mean_merge.head()

,mean_arr_delay
Dep_Airport,
ABE,24.975309
ABQ,-1.442815
ABY,35.102564
ADK,16.285714
ADQ,-5.428571


In [17]:
final = mean_merge.merge(main_df3, how="left", on="Dep_Airport")
final2 = final.drop_duplicates(subset="Dep_Airport")

In [18]:
final2.head()

,Dep_Airport,mean_arr_delay,Mkt_Ccode,Mkt_Fl_No,Arr_Airport,Date,Day_Week,Sched_Dep_Time_OAG,Gate_Dep_Time_Actual,Sched_Arr_Time_OAG,...,City name_x,Country name_x,lat,lng,City name_y,Airport Code,Airport Name_y,Country name_y,Country Abbrev.,World Area Code
0,ABE,24.975309,AA,4177.0,PHL,20180119,fri,620,610,707,...,Allentown,United States,40.652100,-75.440804,Allentown,ABE,Lehigh Valley International,United States,US,67
81,ABQ,-1.442815,WN,462.0,SAN,20180131,wed,1610,1712,1705,...,Albuquerque,United States,35.040199,-106.609001,Albuquerque,ABQ,Albuquerque International,United States,US,67
763,ABY,35.102564,DL,3280.0,ATL,20180213,tue,1759,1750,1907,...,Albany,United States,31.535500,-84.194504,Albany,ABY,Dougherty County,United States,US,67
841,ADK,16.285714,AS,161.0,ANC,20180224,sat,1715,1731,2057,...,Adak Island,United States,51.877998,-176.645996,Adak Island,ADK,Adak Island NS,United States,US,67
848,ADQ,-5.428571,AS,40.0,ANC,20180206,tue,1702,1640,1752,...,Kodiak,United States,57.750000,-152.494003,Kodiak,ADQ,Kodiak Airport,United States,US,67


In [19]:
final2.rename(columns = {"City name_x" : "city_name", "Country name_y": "country_name"}, inplace = True)
final2.dropna(subset=['country_name'])

C:\Users\dlken\anaconda3\envs\PythonData\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Dep_Airport,mean_arr_delay,Mkt_Ccode,Mkt_Fl_No,Arr_Airport,Date,Day_Week,Sched_Dep_Time_OAG,Gate_Dep_Time_Actual,Sched_Arr_Time_OAG,...,city_name,Country name_x,lat,lng,City name_y,Airport Code,Airport Name_y,country_name,Country Abbrev.,World Area Code
0,ABE,24.975309,AA,4177.0,PHL,20180119,fri,620,610,707,...,Allentown,United States,40.652100,-75.440804,Allentown,ABE,Lehigh Valley International,United States,US,67
81,ABQ,-1.442815,WN,462.0,SAN,20180131,wed,1610,1712,1705,...,Albuquerque,United States,35.040199,-106.609001,Albuquerque,ABQ,Albuquerque International,United States,US,67
763,ABY,35.102564,DL,3280.0,ATL,20180213,tue,1759,1750,1907,...,Albany,United States,31.535500,-84.194504,Albany,ABY,Dougherty County,United States,US,67
841,ADK,16.285714,AS,161.0,ANC,20180224,sat,1715,1731,2057,...,Adak Island,United States,51.877998,-176.645996,Adak Island,ADK,Adak Island NS,United States,US,67
848,ADQ,-5.428571,AS,40.0,ANC,20180206,tue,1702,1640,1752,...,Kodiak,United States,57.750000,-152.494003,Kodiak,ADQ,Kodiak Airport,United States,US,67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199514,VLD,0.343750,DL,3383.0,ATL,20180222,thu,1143,1149,1303,...,Valdosta,United States,30.782499,-83.276703,Valdosta,VLD,Regional,United States,US,67
199578,VPS,0.179245,AA,2283.0,DFW,20180214,wed,1326,1321,1543,...,Valparaiso,United States,30.483200,-86.525398,Valparaiso,VPS,Ft. Walton Beach,United States,US,67
199684,WRG,-0.807692,AS,64.0,KTN,20180206,tue,1623,1703,1654,...,Wrangell,United States,56.484299,-132.369995,Wrangell,WRG,Wrangell SPB,United States,US,67
199736,XNA,32.809045,AA,4388.0,LGA,20180129,mon,1734,1758,2129,...,Bentonville,United States,36.281898,-94.306801,Fayetteville,XNA,Northwest Arkansas Rgn,United States,US,67


In [20]:
def df_to_geojson(df, properties, lat='lat', lon='lng'):
    geojson = {'type':'FeatureCollection', 'features':[]}
    for _, row in df.iterrows():
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}
        feature['geometry']['coordinates'] = [row[lon],row[lat]]
        for prop in properties:
            feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
    return geojson

In [21]:
cols = ['Dep_Airport', "city_name", "mean_arr_delay"]
geojson = df_to_geojson(final2, cols)

In [22]:
output_filename = 'airport.geojson'
with open(output_filename, 'w') as json_file:
    json.dump(geojson, json_file)

In [23]:
final2.to_csv('final2.csv')

In [24]:
final2.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 224 entries, 0 to 199935
Data columns (total 43 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Dep_Airport                             224 non-null    object 
 1   mean_arr_delay                          224 non-null    float64
 2   Mkt_Ccode                               224 non-null    object 
 3   Mkt_Fl_No                               224 non-null    float64
 4   Arr_Airport                             224 non-null    object 
 5   Date                                    224 non-null    int64  
 6   Day_Week                                224 non-null    object 
 7   Sched_Dep_Time_OAG                      224 non-null    int64  
 8   Gate_Dep_Time_Actual                    224 non-null    int64  
 9   Sched_Arr_Time_OAG                      224 non-null    int64  
 10  Gate_Arr_Time_Actual                    224 non-null    int